Reference:

https://medium.com/@assafelovic/how-to-build-an-openai-assistant-with-internet-browsing-ee5ad7625661

In [ ]:
!pip install --upgrade openai

In [ ]:
!pip install tavily-python duckduckgo-search

In [ ]:
import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient
from duckduckgo_search import DDGS
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Initialize clients with API keys
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
# Function to perform a Tavily search - Requires API Key
def tavily_search(query):
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=8000)
    return search_result

tavily_search_tool_function = {
    "type": "function",
    "function": {
        "name": "tavily_search",
        "description": "Get information on recent events from the web.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
            },
            "required": ["query"]
        }
    }
}

In [ ]:
# Function to perform a DuckDuckGo search - Do not require API Key.
def duckduckgo_search(query):
      with DDGS() as ddgs:
        return str([r for r in ddgs.text(query, max_results=3)])
      
duckduckgo_search_tool_function = {
    "type": "function",
    "function": {
        "name": "duckduckgo_search",
        "description": "Get information on recent events from the web.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
            },
            "required": ["query"]
        }
    }
}

In [ ]:
# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

In [ ]:
# Function to handle tool output submission
def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        query = json.loads(function_args)["query"]

        if function_name == "tavily_search":
            print(f"Executing query via tavily_search function: {query}")
            output = tavily_search(query=query)

        if function_name == "duckduckgo_search":
            print(f"Executing query via duckduckgo_search function: {query}")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

In [ ]:
# Function to print messages from a thread
def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

In [ ]:
finance_assistant_prompt_instruction = """You are a finance expert. 
Your goal is to provide answers based on information from the internet. 
You must use the provided Tavily search API function to find relevant online information. 
You should never use your own knowledge to answer questions.
Please include relevant url sources in the end of your answers.
"""

In [ ]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="finance_assistant",
    instructions=finance_assistant_prompt_instruction,
    model="gpt-4o",
    tools=[duckduckgo_search_tool_function]
)

assistant_id = assistant.id
print(f"Assistant ID: {assistant_id}")

In [ ]:
# Create a thread
thread = client.beta.threads.create()
print(f"Thread: {thread}")

In [ ]:
# Ongoing conversation loop
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break

    # Create a message
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )
    print(f"Run ID: {run.id}")

    # Wait for run to complete
    run = wait_for_run_completion(thread.id, run.id)

    if run.status == 'failed':
        print(run.error)
        continue
    elif run.status == 'requires_action':
        run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
        run = wait_for_run_completion(thread.id, run.id)

    # Print messages from the thread
    print_messages_from_thread(thread.id)